In [50]:
from PyQt5.QtCore import QObject, pyqtSignal

class MyObject(QObject):
    my_signal = pyqtSignal(int)

    def some_method(self):
        # Emitting the signal with a string and an integer argument
        self.my_signal.emit(42)

# Usage
obj = MyObject()
obj.my_signal.connect(lambda i: print(f"Received: {i}"))
obj.some_method()


Received: 42


In [ ]:
#fix error: qt.gui.icc: fromIccProfile: failed minimal tag size sanity
#stackoverflow Q 65463848
#careful as better monitors are capable of color space better that sRGB 
#and converting all images into sRGB may take away from color ranges
#make it go through 1 pass while classifying images
import io
from PIL import Image, ImageCms

def convert_to_srgb(file_path):
    """
    Convert PIL image to sRGB color space (if possible)
    and save the converted file.
    """
    img = Image.open(file_path)
    icc = img.info.get('icc_profile', '')
    icc_conv = ''

    if icc:
        io_handle = io.BytesIO(icc)     # virtual file
        src_profile = ImageCms.ImageCmsProfile(io_handle)
        dst_profile = ImageCms.createProfile('sRGB')
        img_conv = ImageCms.profileToProfile(img, src_profile, dst_profile)
        icc_conv = img_conv.info.get('icc_profile','')

    if icc != icc_conv:
        # ICC profile was changed -> save converted file
        img_conv.save(file_path,
                    format = 'JPEG',
                    quality = 100,
                    icc_profile = icc_conv)
        
files=glob.glob(f"/media/mgama1/mgama1/photos/**/*.jpeg",recursive=True)
for file in files:
    convert_to_srgb(file)

In [ ]:
!qta-browser

# no lazy loading no thumbnail

In [ ]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
import glob
import time
import sys
import os
s_t=time.time()
class ImageThumbnailWidget(QWidget):
    def __init__(self, image_path):
        super().__init__()

        self.image_path = image_path
        self.bg_color="#222324"
        
        self.init_ui()
    def init_ui(self):
        layout = QVBoxLayout()
        self.setStyleSheet(f"background-color: {self.bg_color};")

        pixmap = QPixmap(self.image_path)
        pixmap = pixmap.scaledToWidth(200)  

        self.label = QLabel()
        self.label.setPixmap(pixmap)
        self.label.setAlignment(Qt.AlignCenter)

        layout.addWidget(self.label)
        self.setLayout(layout)

    def enterEvent(self, event):
        # Change background color when the mouse enters
        self.setStyleSheet("background-color: #606162;")

    def leaveEvent(self, event):
        # Reset background color when the mouse leaves
        self.setStyleSheet(f"background-color: {self.bg_color};")

    def mousePressEvent(self, event):
        self.setStyleSheet("background-color: #0f68db;")
    
    def mouseReleaseEvent(self, event):
            self.setStyleSheet(f"background-color: {self.bg_color};")
            
            
            
            
class ImageGalleryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.bg_color="#222324"
        self.init_ui()

    def init_ui(self):
        central_widget = QWidget()
        self.setStyleSheet(f"background-color: {self.bg_color};")

        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)

        scroll_content = QWidget(scroll_area)
        scroll_area.setWidget(scroll_content)

        layout = QGridLayout(scroll_content)

        #current_directory = os.getcwd()
        file_types=['jpg','jpeg','png']
        image_files=[]
        for file_type in file_types:
            image_files+=(glob.glob(f"/media/mgama1/mgama1/photos/**/*.{file_type}",recursive=True))

        row, col = 0, 0
        for index, image_file in enumerate(image_files):
            
            thumbnail_widget = ImageThumbnailWidget(image_file)
            layout.addWidget(thumbnail_widget, row, col)
            col += 1

            if col == round(self.width()/200):
                col = 0
                row += 1

        central_widget.setLayout(layout)
        scroll_area.setWidget(central_widget)

        self.setCentralWidget(scroll_area)

        self.setGeometry(100, 100, 800, 600)
        self.setWindowTitle('OGallery')
        self.show()
        print(time.time()-s_t)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    gallery_app = ImageGalleryApp()
    sys.exit(app.exec_())
#6.7s 5.3s 5.2s 5.2s 5.2s

# thumbnail using cache

In [ ]:
#linux thumbnail metadata
from PIL import Image
import hashlib

filename = '/home/mgama1/.cache/thumbnails/large/0abccf13363918397280761f6766f230.png'
img = Image.open(filename)
print("linux: \n",img.info)
filename = '/home/mgama1/.cache/OpenGallery/0abccf13363918397280761f6766f230.png'

img = Image.open(filename)
print("OG: \n",img.info)

In [3]:
from custom import *
pixmap1 = QPixmap(qta.icon('mdi.jellyfish',color='#faf7f7').pixmap(400,400))
pixmap2 = QPixmap(qta.icon('mdi.fullscreen-exit',color='#e2172b').pixmap(200,200))
combined_pixmap = CustomAwesome().concat_pixmaps(pixmap1, pixmap2,2)
combined_pixmap.save('angry.png')


pixmap1 = QPixmap(qta.icon('mdi.jellyfish',color='#faf7f7').pixmap(400,400))
pixmap2 = QPixmap(qta.icon('mdi.glasses',color='#000000').pixmap(200,200))

combined_pixmap=CustomAwesome().concat_pixmaps(pixmap1, pixmap2,3)
combined_pixmap.save('info.png')


pixmap1 = QPixmap(qta.icon('mdi.jellyfish',color='#faf7f7').pixmap(400,400))
pixmap2 = QPixmap(qta.icon('fa.exclamation',color='#fde01a').pixmap(200,200))
combined_pixmap = CustomAwesome().concat_pixmaps(pixmap1, pixmap2,1.5)
combined_pixmap.save('warning.png')

True

In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
import glob
import sys
import os


class ImageThumbnailWidget(QWidget):
    def __init__(self, image_path):
        super().__init__()
        self.image_path = image_path
        self.bg_color = "#222324"
        self.init_ui()

    def init_ui(self):
        layout = QVBoxLayout()
        self.setStyleSheet(f"background-color: {self.bg_color};")

        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)

        layout.addWidget(self.label)
        self.setLayout(layout)

    def load_thumbnail(self):
        pixmap = QPixmap(self.image_path)
        pixmap = pixmap.scaledToWidth(200)
        self.label.setPixmap(pixmap)

    def enterEvent(self, event):
        # Change background color when the mouse enters
        self.setStyleSheet("background-color: #606162;")

    def leaveEvent(self, event):
        # Reset background color when the mouse leaves
        self.setStyleSheet(f"background-color: {self.bg_color};")


class ImageGalleryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.bg_color = "#222324"
        self.scroll_value =   0  # Initialize scroll_value
        self.batch_size =   3  # Number of thumbnails to load per batch
        self.loaded_count =   0  # Counter for thumbnails loaded in the current batch
        self.scrollbar_threshold =   600  # Scrollbar threshold for loading thumbnails
        self.thumbnails_to_load = []  # List of thumbnails to load
        self.init_ui()

    def init_ui(self):
        central_widget = QWidget()
        self.setStyleSheet(f"background-color: {self.bg_color};")

        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)

        scroll_content = QWidget(scroll_area)
        scroll_area.setWidget(scroll_content)

        layout = QGridLayout(scroll_content)

        file_types = ['jpg', 'jpeg', 'png']
        image_files = []
        for file_type in file_types:
            image_files += (glob.glob(f"/home/mgama1/.cache/OpenGallery/**/*.{file_type}", recursive=True))

        row, col =   0,   0
        for index, image_file in enumerate(image_files):
            thumbnail_widget = ImageThumbnailWidget(image_file)
            layout.addWidget(thumbnail_widget, row, col)
            col +=   1

            if col == round(self.width() /   200):
                col =   0
                row +=   1

            self.thumbnails_to_load.append(thumbnail_widget)

        central_widget.setLayout(layout)
        scroll_area.setWidget(central_widget)

        self.setCentralWidget(scroll_area)

        self.scroll = scroll_area.verticalScrollBar()
        self.scroll.valueChanged.connect(self.update_thumbnails)

        self.setGeometry(300, 100, 750, 550)
        self.setWindowTitle('OGallery')
        self.show()

        # Load the first batch of thumbnails immediately
        self.load_initial_batch()

    def load_initial_batch(self):
        for i in range(9):
            self.thumbnails_to_load[i].load_thumbnail()
            self.loaded_count +=   1
        del self.thumbnails_to_load[:self.batch_size]

    def update_thumbnails(self):
        self.scroll_value = self.scroll.value()
        if self.scroll_value >= self.scrollbar_threshold or self.thumbnails_to_load:
            for i in range(min(len(self.thumbnails_to_load), self.batch_size)):
                self.thumbnails_to_load[i].load_thumbnail()
                self.loaded_count +=   1
            del self.thumbnails_to_load[:self.batch_size]
            self.scrollbar_threshold += self.scrollbar_threshold


if __name__ == '__main__':
    app = QApplication(sys.argv)
    gallery_app = ImageGalleryApp()
    sys.exit(app.exec_())


SystemExit: 0

/home/mgama1/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def crop(image,x=0,y=0,w=200 ,h=200):
    return image[y:y+h, x:x+w]


In [ ]:
import glob
dirfilesnames=(glob.glob("./cats/*"))
hash1 = imagehash.average_hash(Image.open('cat.jpg'))

hashdic={}
for filename in dirfilesnames:
    hashdic.update({filename:imagehash.average_hash(Image.open(filename))})

for hkey in hashdic:
    if (hash1-hashdic[hkey])<34:
        print(hkey)
    


In [ ]:
import numpy as np
import plotly.graph_objects as go
from matplotlib.colors import rgb2hex
#I don't know about you but i don't wanna plot the whole 286,832 pixel
#so let's just take a sample
sample_indices = np.random.choice(np.arange(pixels.shape[0]), size=2000, replace=False)
sampled_pixels = pixels[sample_indices]
points = sampled_pixels  

# Create a colormap based on RGB values
colors = [rgb2hex(rgb / 255) for rgb in points]

# Format RGB values as strings
rgb_strings = [f"R: {int(rgb[0])}, G: {int(rgb[1])}, B: {int(rgb[2])}" for rgb in points]

# Create a scatter plot with smaller points, colored by RGB values, and RGB values as hover text
fig = go.Figure(data=[go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(size=3, color=colors),  # Color points based on RGB values
    text=rgb_strings,  # Use RGB values as hover text
)])

fig.update_layout(scene=dict(
    xaxis_title='R',
    yaxis_title='G',
    zaxis_title='B')
    )

fig.show()


In [ ]:
import rembg
import cv2
import numpy as np
img=cv2.imread("../Cat_August_2010-4.jpg")
mask = rembg.remove(img,only_mask=True)
mask_3c=cv2.merge([mask,mask,mask])

# Read the images
foreground = np.copy(img).astype(float)
background = cv2.blur(img,(9,9)).astype(float)
alpha = mask_3c

# Normalize the alpha mask to keep intensity between 0 and 1
alpha = alpha.astype(float)/255

# Multiply the foreground with the alpha matte
foreground = cv2.multiply(alpha, foreground)/255

# Multiply the background with ( 1 - alpha )
background = cv2.multiply(1.0 - alpha, background)/255

# Add the masked foreground and background.
outImage = cv2.add(foreground, background)
outImage = (outImage * 255).astype(np.uint8)

display(outImage,100)


In [3]:
def display(image,scale_percent=50):
    
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
  
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    cv2.imshow('display', resized) 
    cv2.waitKey(0)   
    cv2.destroyAllWindows()

# phase 1.5

- select multiple to del



# phase 2
- text ocr
- object detection and decoding of qr code
- equations to latex
- Encrypted folder

# phase 3
- line_profiler and memory_profiler
- better models:  multilabel classification - Blip Image Captioning - one shot learning (needs a lot of convineint data)

In [ ]:
import subprocess

def set_background(image_path):
    command = ["gsettings", "set", "org.gnome.desktop.background", "picture-uri", f"{image_path}"]
    subprocess.run(command, check=True)

image_path = "/media/mgama1/mgama1/photos/190529-N-XD935-010X.jpeg"
set_background(image_path)


In [ ]:
import os

def set_background(image_path):
    command = f"gsettings set org.gnome.desktop.background picture-uri file://{image_path}"
    os.system(command)
    
# Example usage
set_background(image_path)


SystemExit: 0

/home/mgama1/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
